In [2]:
import os ,csv, boto3
import numpy as np
from sklearn import preprocessing, neighbors, model_selection, tree, ensemble
import pandas as pd

In [11]:
df = pd.read_csv(r'C:\Users\Sonal Kumari\Documents\Image_Classification-master\Data5.csv')
df.drop('Image name', axis =1, inplace = True)
df.head()

,Unnamed: 0,Actual Room,Bedroom,Dorm Room,Bed,Rug,Cushion,Pillow,Furniture,Bathroom,...,Housing,Chair,Urban,Living Room,Couch,Transportation,Outdoors,Tree,Plant,Table
0,0,Bathroom,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Bathroom,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Bathroom,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,3,Bathroom,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Bathroom,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = np.array(df.drop(['Actual Room'],1))
y = np.array(df['Actual Room'])
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)


In [16]:
clf = neighbors.KNeighborsClassifier()
#clf = tree.DecisionTreeClassifier()
#clf = ensemble.RandomForestClassifier()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.9977011494252873


In [23]:
#setup
os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'
keys = pd.read_csv(r'C:\Users\Sonal Kumari\Documents\Image_Classification-master\Keys.csv')
access_key_id = keys['aws access key'][0]
secret_key = keys['aws secret key'][0]
client = boto3.client('rekognition', aws_access_key_id = access_key_id, aws_secret_access_key = secret_key)
s3_client = boto3.client('s3', aws_access_key_id = access_key_id, aws_secret_access_key = secret_key)


In [18]:
#convert response to acceptable input value
threshold_confidence_for_identification = 80.0
def getInstanceArray(res):
    instance_array = np.zeros(len(df.columns)-1)
    for item in res['Labels']:
        try:
            index = df.columns.get_loc(item['Name'])
            instance_array[index-1] = int(item['Confidence']/threshold_confidence_for_identification)
        except KeyError:
            continue
    return instance_array


In [21]:

#classify response  #prediction_prob = [bathroom_prob, bedroom_prob, kitchen_prob, other]
response = client.detect_labels(Image = {"S3Object": {"Bucket":'test-bucket-sonal', 
                                                      "Name":'check/kitchen1.jpg'}})

example_measures = getInstanceArray(response)
example_measures = example_measures.reshape(1, -1)
prediction = clf.predict(example_measures)
print(prediction)
clf.predict_proba(example_measures)

['Bathroom']


array([[1., 0., 0., 0.]])